In [1]:
from tkinter import *
import pandas as pd
from sklearn.model_selection import train_test_split
from KMean_Clustering import KMean_Clustering
from sklearn.metrics import davies_bouldin_score, silhouette_score

### Đọc file dữ liệu đầu vào

In [2]:
data = pd.read_csv(r"heart_disease_patients.csv")
data.head()

,id,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope
0,1,63,1,1,145,233,1,2,150,0,2.3,3
1,2,67,1,4,160,286,0,2,108,1,1.5,2
2,3,67,1,4,120,229,0,2,129,1,2.6,2
3,4,37,1,3,130,250,0,0,187,0,3.5,3
4,5,41,0,2,130,204,0,2,172,0,1.4,1


### Chia dữ liệu thành 2 phần train:test tỉ lệ 90:10
##### Lấy từ cột 1 trở đi (không lấy cột id)

In [3]:
dtTrain, dtTest = train_test_split(data, test_size=0.1, shuffle=False)
X_Train = dtTrain.iloc[:,1:]
X_Test = dtTest.iloc[:,1:]
X_Train = X_Train.values
X_Test = X_Test.values

### Tìm ra model tốt nhất
##### Model tốt nhất là model có độ đo silhoutte gần 1

In [4]:
maxx = 0
bestmodel = 0
for i in range(2,10):
    model = KMean_Clustering(K=i)
    (centroids, labels, it) = model.fit(X_Train)
    y_pred = model.predict(X_Test)
    if (silhouette_score(X_Test, y_pred[-1])>maxx): 
        maxx = silhouette_score(X_Test, y_pred[-1])
        k = i
        bestmodel = model

### Train mô hình tốt nhất vừa tìm được

In [5]:
(centroids, labels, it) = bestmodel.fit(X_Train)
print("So cum tot nhat la: ", k)
print("Cac tam cum la:\n", centroids[-1])

So cum tot nhat la:  2
Cac tam cum la:
 [[5.68787879e+01 5.55555556e-01 3.23232323e+00 1.36000000e+02
  3.00787879e+02 1.51515152e-01 1.20202020e+00 1.45686869e+02
  3.83838384e-01 1.04545455e+00 1.58585859e+00]
 [5.28843931e+01 7.51445087e-01 3.13872832e+00 1.28745665e+02
  2.20000000e+02 1.56069364e-01 9.30635838e-01 1.52184971e+02
  3.00578035e-01 1.02716763e+00 1.57803468e+00]]


### Dự đoán nhãn cho tập X_Test (10% dữ liệu còn lại)

In [6]:
labels_pred = bestmodel.predict(X_Test)
print(labels_pred[-1])

[0 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1]


### Các độ đo đánh giá mô hình

In [7]:
print("davies_boudlin: ", davies_bouldin_score(X_Test, labels_pred[-1]))
print("silhouette: ", silhouette_score(X_Test, labels_pred[-1]))

davies_boudlin:  0.672308432300016
silhouette:  0.5001822850641424


## Tạo ứng dụng dự đoán nhãn của 1 mẫu bất kỳ

In [8]:
# Tạo cửa sổ chính của ứng dụng GUI
form = Tk()
form.title("Phân cụm người mắc bệnh tim")
form.geometry("350x580")

''

In [9]:
# Tạo các label và textbox
labels = ['age','sex','cp','trestbps','chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope']
entry_list = []
for i, label_text in enumerate(labels):
    label = Label(form, text=label_text)
    label.grid(row=i, column=0, padx=15, pady=10)
    entry = Entry(form)
    entry.grid(row=i, column=1, padx=15, pady=10)
    entry_list.append(entry)

In [10]:
# Tạo label để hiển thị nhãn phân cụm dự đoán và độ đo chất lượng mô hình
label_text = StringVar()
label_result = Label(form, textvariable=label_text)
label_result.grid(row=510, column=0, columnspan=2, padx=10, pady=10)

evaluation_text = StringVar()
evaluation_result = Label(form, textvariable=evaluation_text)
evaluation_result.grid(row=520, column=0, columnspan=2, padx=10, pady=10)

In [11]:
# Hàm để dự đoán nhãn phân cụm của mẫu mới và tính độ đo chất lượng mô hình
def predict_cluster():
    # Lấy dữ liệu từ ô textbox
    features = []
    for entry in entry_list:
        features.append(float(entry.get()))
    
    # Thực hiện dự đoán nhãn phân cụm
    predicted_label = bestmodel.predict([features])
    
    # Hiển thị nhãn phân cụm
    label_text.set("Nhãn phân cụm: " + str(predicted_label[0]))
    
    # Tính độ đo chất lượng mô hình
    labels_pred = bestmodel.predict(X_Test)
    silhouette = silhouette_score(X_Test, labels_pred[-1])
    davies_bouldin = davies_bouldin_score(X_Test, labels_pred[-1])
    
    # Hiển thị độ đo chất lượng mô hình
    evaluation_text.set("Silhouette: {:.2f}, Davies-Bouldin: {:.2f}".format(silhouette, davies_bouldin))

In [12]:
# Tạo nút dự đoán và đặt sự kiện khi click vào nút
predict_button = Button(form, text="Dự đoán", command=predict_cluster)
predict_button.grid(row=500, column=0, columnspan=2, padx=10, pady=10)

In [13]:
# Gọi vòng lặp sự kiện chính để các hành động có thể diễn ra trên màn hình máy tính của người dùng
form.mainloop()